In [1]:
from ConfigSpace import Configuration, ConfigurationSpace

import numpy as np
from smac import HyperparameterOptimizationFacade, Scenario
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import subprocess

iris = datasets.load_iris()

In [2]:
def train(config: Configuration, seed: int = 0) -> float:
    classifier = SVC(C=config["C"], random_state=seed)
    scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
    return 1 - np.mean(scores)


configspace = ConfigurationSpace({"C": (0.100, 1000.0)})

# Scenario object specifying the optimization environment
scenario = Scenario(configspace, deterministic=True, n_trials=300)

# Use SMAC to find the best configuration/hyperparameters
smac = HyperparameterOptimizationFacade(scenario, train)

[INFO][abstract_initial_design.py:147] Using 10 initial design configurations and 0 additional configurations.


In [3]:
incumbent = smac.optimize()

[INFO][abstract_intensifier.py:306] Using only one seed for deterministic scenario.
[INFO][abstract_intensifier.py:516] Added config 7707e1 as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:595] Added config a3ff11 and rejected config 7707e1 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:595] Added config 5755d7 and rejected config a3ff11 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][abstract_intensifier.py:595] Added config db9b45 and rejected config 5755d7 as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:320] Finished 50 trials.
[INFO][smbo.py:320] Finished 100 trials.
[INFO][smbo.py:320] Finished 150 trials.
[INFO][smbo.py:320] Finished 200 trials.
[INFO][smbo.py:320] Finished 250 trials.
[INFO][smbo.py:320] Finished 300 trials.
[INFO][smbo.py:328] Configuration budget is exhausted:
[INFO][smbo.py:329] --- Remaining w

In [4]:
incumbent

Configuration(values={
  'C': 7.8889654918543,
})

In [8]:
# run program LKH-2.0.10\LKH-2.exe with an argument config.par as a subprocess and capture the output, where there is a line "Time.total = 7.09 sec." with the time in seconds, return the time

result = subprocess.run(['LKH-2.0.10\LKH-2.exe', 'config.par'], 
                       capture_output=True, 
                       text=True,
                       stdin=subprocess.DEVNULL)

for line in result.stdout.splitlines():
    if "Time.total" in line:
        time = float(line.split()[-2])
        break

time


5.95

In [9]:
print(result.stdout)


PARAMETER_FILE = config.par
PROBLEM_FILE = CEPS/instance_set/mutator_TSP/cluster/1.tsp
Successes/Runs = 0/1
Cost.min = 17206398, Cost.avg = 17206398.00, Cost.max = 17206398
Gap.min = 0.0000%, Gap.avg = 0.0000%, Gap.max = 0.0000%
Trials.min = 800, Trials.avg = 800.0, Trials.max = 800
Time.min = 5.27 sec., Time.avg = 5.27 sec., Time.max = 5.27 sec.
Time.total = 5.95 sec.
Press any key to continue . . . 



In [4]:
from ConfigSpace import Configuration, ConfigurationSpace, Float, Integer

from smac import HyperparameterOptimizationFacade, Scenario
from smac.runhistory.dataclasses import TrialValue

class TSP:
    @property
    def configspace(self) -> ConfigurationSpace:
        cs = ConfigurationSpace(seed=0)
        patching_c = Integer("patching_c", (1, 5), default=2)
        patching_a = Integer("patching_a", (1, 5), default=2)
        cs.add([patching_c, patching_a])

        return cs

    def train(self, config: Configuration, seed: int = 0) -> float:
        patching_c = config["patching_c"]
        patching_a = config["patching_a"]
        with open('config.par', 'w') as f:
            f.write(f"""PROBLEM_FILE = CEPS/instance_set/mutator_TSP/cluster/0.tsp
        MOVE_TYPE = 5
        PATCHING_C = {patching_c}
        PATCHING_A = {patching_a}
        RUNS = 1
        OUTPUT_TOUR_FILE = best
        TOTAL_TIME_LIMIT = 100
        TRACE_LEVEL = 0""")

        result = subprocess.run(['LKH-2.0.10\LKH-2.exe', 'config.par'], 
                            capture_output=True, 
                            text=True,
                            stdin=subprocess.DEVNULL)

        for line in result.stdout.splitlines():
            if "Time.total" in line:
                time = float(line.split()[-2])
                break
        return time


model = TSP()
scenario = Scenario(model.configspace, deterministic=False, n_trials=3)

intensifier = HyperparameterOptimizationFacade.get_intensifier(
    scenario,
    max_config_calls=1,  # We basically use one seed per config only
)

smac = HyperparameterOptimizationFacade(
    scenario,
    model.train,
    intensifier=intensifier,
    overwrite=True,
)

for _ in range(3):
    info = smac.ask()
    assert info.seed is not None

    cost = model.train(info.config, seed=info.seed)
    value = TrialValue(cost=cost, time=0.5)
    print(f"Cost: {cost}, Time: {value.time}")
    smac.tell(info, value)

# After calling ask+tell, we can still optimize
# Note: SMAC will optimize the next 90 trials because 10 trials already have been evaluated
incumbent = smac.optimize()

# Get cost of default configuration
default_cost = smac.validate(model.configspace.get_default_configuration())
print(f"Default cost: {default_cost}")

# Let's calculate the cost of the incumbent
incumbent_cost = smac.validate(incumbent)
print(f"Incumbent cost: {incumbent_cost}")

[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 20 to 1 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 1 initial design configurations and 0 additional configurations.
Cost: 7.28, Time: 0.5
[INFO][abstract_intensifier.py:516] Added config a7da4c as new incumbent because there are no incumbents yet.
Cost: 8.69, Time: 0.5
Cost: 8.72, Time: 0.5
[INFO][abstract_intensifier.py:595] Added config b4d80f and rejected config a7da4c as incumbent because it is not better than the incumbents on 1 instances:
[INFO][smbo.py:328] Configuration budget is exhausted:
[INFO][smbo.py:329] --- Remaining wallclock time: inf
[INFO][smbo.py:330] --- Remaining cpu time: inf
[INFO][smbo.py:331] --- Remaining trials: -1
Default cost: 8.75
Incumbent cost: 6.33
